1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
value = input("Insert the value you want to convert:")
base = input("Insert the base of this number:")

def converter(val,n):
    
    if n==2:
        case = int(val,n)
        print("In base 10:",case)#base10
        print("In base 16:",hex(case))#base16
    elif n==16:
        case = int(val,n)
        print("In base 10:",case)#base10
        print("In base 2:",bin(case))#base2    
    elif n==10: 
        print("In base 16:",hex(int(val)))
        print("In base 2:",bin(int(val))) 
    else:
        print("-----BASE NOT VALID-----")

    return
converter(value,int(base))


Insert the value you want to convert:1101
Insert the base of this number:2
In base 10: 13
In base 16: 0xd


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
print("Given the 32-digit value: 0|10001100|00101100000000000111000")

ieee_32 = '0|10001100|00101100000000000111000'

def convertToInt(mantissa_str): 
    power_count = -1
    mantissa_int = 0
    for i in mantissa_str: 
        mantissa_int += (int(i) * pow(2, power_count)) 
        power_count -= 1   
    return (mantissa_int + 1) 
  
sign_bit = int(ieee_32[0]) 
exponent_bias = int(ieee_32[2 : 10], 2) 
exponent_unbias = exponent_bias - 127
mantissa_str = ieee_32[11 : ] 

mantissa_int = convertToInt(mantissa_str) 
real_no = pow(-1, sign_bit) * mantissa_int * pow(2, exponent_unbias) 

print("The relative float value is :",real_no) 

Given the 32-digit value: 0|10001100|00101100000000000111000
The relative float value is : 9600.0546875


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
under=1
over=1.
N_over=1024
N_under=1075

for i in range(1,N_over):
    over = over*2

for i in range(1,N_under):
    under=under/2
    
print(over) 
print(under)

print("for the value under the underflow limit is 5e-324 at N=1075")
print("for the value over the overflow limit is about 9e+307 at N=1024")

8.98846567431158e+307
5e-324
for the value under the underflow limit is 5e-324 at N=1075
for the value over the overflow limit is about 9e+307 at N=1024


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
old=1
new_=1

for i in range(1,1000):
    new_ = new_/2
    new = old + new_
    if new == 1:
        break
    value = new#the value before the last that breaks the cycle
    
print("The value N for which they are the same is:",i)
print("The minimum value we can add is:",value)


The value N for which they are the same is: 53
The minimum value we can add is: 1.0000000000000002


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [5]:
#a)
from math import sqrt
a = 0.001 
b = 1000
c = 0.001

def function1(a,b,c):

    x1 = (-b + sqrt(b**2 -4*a*c))/(2*a)
    x2 = (-b - sqrt(b**2 -4*a*c))/(2*a)

    return [x1,x2]

print("for a) the functions returns:",function1(a,b,c))

#b)
def function2(a,b,c):
    
    x1 = (2*c)/(-b - sqrt(b**2 - 4*a*c))
    x2 = (-2*c)/(-b + sqrt(b**2 + 4*a*c))
    
    return [x1,x2]

print("for b) the functions returns:",function2(a,b,c))
#The result show a better value for x1 but a worst value for x2.
#This is due to the very small numbers involved in the calculation of x2

#c)
def function3(a,b,c):
    
    x1 = (2*c)/(-b -sqrt(b**2 -4*a*c))
    x2 = (-b -sqrt(b**2 -4*a*c))/(2*a)
    
    return [x1,x2]
    
print("for c) the functions returns:",function3(a,b,c))
#This functions takes the best results of the functions above. x1 from function2, x2 from function1.
#From this we have a function that works with any nymber we give it in input.


for a) the functions returns: [-9.999894245993346e-07, -999999.999999]
for b) the functions returns: [-1.000000000001e-06, -1000010.5755125057]
for c) the functions returns: [-1.000000000001e-06, -999999.999999]


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
#a)
delta = 0.01
x = 1

def derivative(x,d):
    
    f = x*(x-1)
    f_d = (x+d)*(x+d-1)
    m = (f_d-f)/d
    
    return m

derivative(x,delta)

err = abs(derivative(x,delta)-1)
print("The accuracy for delta="+ str(delta) +" is:",err) 
print("the result is actually different due to the fact that the mathematical limit has to be interpreted infinitesimal, so the computational calculus can't be perfectly the same.")
      #Obviosly the mathematical definition of limit has to be interpreted infinitesimal 
#b)
deltas = [1e-4, 1e-6, 1e-8, 1e-10, 1e-12, 1e-14]
accuracy = []

def derivative2(x,deltas):
    
    for i in deltas:
        
        f = x*(x-1)
        f_d = (x+i)*(x+i-1)
        accuracy.append(abs(1-((f_d -f)/i)))
        
    return accuracy

print("The relative accuracies for delta smaller and smaller are:")
print(derivative2(x,deltas))
#defining accuracy just by the absolute value of the (f(x+delta)-f(x))/delta:
print("It seems that accuracy scales linearly for the first two delta, then it varies due to the operations with very small numbers involved")

The accuracy for delta=0.01 is: 0.010000000000000897
the result is actually different due to the fact that the mathematical limit has to be interpreted infinitesimal, so the computational calculus can't be perfectly the same.
The relative accuracies for delta smaller and smaller are:
[9.999999988985486e-05, 9.99917733279787e-07, 3.922528746258536e-09, 8.284037100736441e-08, 8.890058334132256e-05, 0.0007992778373491216]
It seems that accuracy scales linearly for the first two delta, then it varies due to the operations with very small numbers involved


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
from math import sqrt, pi
import time

#a)
def integral(N):

    h = 2/N
    x_k = []
    for i in range(N):
        x_k.append(-1 + h*i)
    
    y_k = []
    for i in range(N):
        y_k.append(sqrt(1-x_k[i]**2))
    
    sum = 0
    for i in range(N):
        sum +=h*y_k[i]
        
    accuracy=abs(sum - (pi/2))
    print(accuracy)
    
    return
print("The result for N=100 is",integral(100))
        
#b)
start_time = time.time()
print("The result for N=10e6 is",integral(500000))
print("--- %s seconds ---" % (time.time() - start_time))

print("N = 10e6 is the maximux value for a simulation less than a second(about 0.95 seconds)")
print("for N = 10e8 the simulation runs for about 2 minutes and we have an accuracy of 1e-12")

0.0016620712456461018
The result for N=100 is None
4.703899714186832e-09
The result for N=10e6 is None
--- 0.9961798191070557 seconds ---
N = 10e6 is the maximux value for a simulation less than a second(about 0.95 seconds)
for N = 10e8 the simulation runs for about 2 minutes and we have an accuracy of 1e-12
